# Thanks


https://www.kaggle.com/hirayukis/lightgbm-keras-and-4-kfold

https://www.kaggle.com/marcovasquez/basic-nlp-with-tensorflow-and-wordcloud

https://www.kaggle.com/vbmokin/ion-switching-advanced-fe-lgb-xgb-confmatrix

https://www.kaggle.com/martxelo/fe-and-ensemble-mlp-and-lgbm

https://www.kaggle.com/nxrprime/wavenet-with-shifted-rfc-proba-and-cbr

https://www.kaggle.com/marcovasquez/basic-nlp-with-tensorflow-and-wordcloud

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Any results you write to the current directory are saved as output.
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.preprocessing import RobustScaler, MinMaxScaler

from tqdm import tqdm

import lightgbm as lgb
import xgboost as xgb
import joblib

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight

from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import roc_curve, auc, accuracy_score, cohen_kappa_score
from sklearn.metrics import mean_squared_error, f1_score, confusion_matrix

## Utils

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        if col != 'time':
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def get_stats(df):
    stats = pd.DataFrame(index=df.columns, columns=['na_count', 'n_unique', 'type', 'memory_usage'])
    for col in df.columns:
        stats.loc[col] = [df[col].isna().sum(), df[col].nunique(dropna=False), df[col].dtypes, df[col].memory_usage(deep=True, index=False) / 1024**2]
    stats.loc['Overall'] = [stats['na_count'].sum(), stats['n_unique'].sum(), None, df.memory_usage(deep=True).sum() / 1024**2]
    return stats

def print_header():
    print('col         conversion        dtype    na    uniq  size')
    print()
    
def print_values(name, conversion, col):
    template = '{:10}  {:16}  {:>7}  {:2}  {:6}  {:1.2f}MB'
    print(template.format(name, conversion, str(col.dtypes), col.isna().sum(), col.nunique(dropna=False), col.memory_usage(deep=True, index=False) / 1024 ** 2))
    
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #tf.random.set_seed(seed)    

In [ ]:
def display_set(df, column, n_sample, figsize ):
    f, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = figsize )
    sns.lineplot(x= df.index[::n_sample], y = df[column][::n_sample], ax=ax1)


In [ ]:
def plot_cm(y_true, y_pred, title):
    figsize=(14,14)
    y_pred = y_pred.astype(int)
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    plt.title(title)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)

In [ ]:
def get_class_weight(classes, exp=1):
    '''
    Weight of the class is inversely proportional to the population of the class.
    There is an exponent for adding more weight.
    '''
    hist, _ = np.histogram(classes, bins=np.arange(12)-0.5)
    class_weight = hist.sum()/np.power(hist, exp)
    
    return class_weight

In [ ]:
# Thanks to https://www.kaggle.com/siavrez/simple-eda-model
def MacroF1Metric(preds, dtrain):
    labels = dtrain.get_label()
    preds = np.round(np.clip(preds, 0, 10)).astype(int)
    score = f1_score(labels, preds, average = 'macro')
    return ('MacroF1Metric', score, True)

def multiclass_F1_Metric(preds, dtrain):
    labels = dtrain.get_label()
    num_labels = 11
    preds = preds.reshape(num_labels, len(preds)//num_labels)
    preds = np.argmax(preds, axis=0)
    score = f1_score(labels, preds, average="macro")
    return ('MacroF1Metric', score, True)

In [ ]:
from functools import partial
import scipy as sp
class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize F1 (Macro) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """
    def __init__(self):
        self.coef_ = 0

    def _f1_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients
        
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

        return -f1_score(y, X_p, average = 'macro')

    def fit(self, X, y):
        """
        Optimize rounding thresholds
        
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._f1_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])


    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']
    
    
def optimize_predictions(prediction, coefficients):
    prediction[prediction <= coefficients[0]] = 0
    prediction[np.where(np.logical_and(prediction > coefficients[0], prediction <= coefficients[1]))] = 1
    prediction[np.where(np.logical_and(prediction > coefficients[1], prediction <= coefficients[2]))] = 2
    prediction[np.where(np.logical_and(prediction > coefficients[2], prediction <= coefficients[3]))] = 3
    prediction[np.where(np.logical_and(prediction > coefficients[3], prediction <= coefficients[4]))] = 4
    prediction[np.where(np.logical_and(prediction > coefficients[4], prediction <= coefficients[5]))] = 5
    prediction[np.where(np.logical_and(prediction > coefficients[5], prediction <= coefficients[6]))] = 6
    prediction[np.where(np.logical_and(prediction > coefficients[6], prediction <= coefficients[7]))] = 7
    prediction[np.where(np.logical_and(prediction > coefficients[7], prediction <= coefficients[8]))] = 8
    prediction[np.where(np.logical_and(prediction > coefficients[8], prediction <= coefficients[9]))] = 9
    prediction[prediction > coefficients[9]] = 10
    
    return prediction    

## Load train and test datasets

**IMPORTANT: While the time series appears continuous, the data is from discrete batches of 50 seconds long 10 kHz samples (500,000 rows per batch). In other words, the data from 0.0001 - 50.0000 is a different batch than 50.0001 - 100.0000, and thus discontinuous between 50.0000 and 50.0001.**

In [ ]:
RANDOM_SEED = 42
GROUP_BATCH_SIZE2 = 80000
GROUP_BATCH_SIZE1 = 20000
WINDOW_SIZES = [3, 5, 10, 50, 100, 500, 1000, 5000]

## 3, 5, 2000, 5000

seed_everything(RANDOM_SEED)

In [ ]:
PATH = '/kaggle/input/data-without-drift/'
#PATH = '/kaggle/input/liverpool-ion-switching/'

train = pd.read_csv(PATH + 'train_clean.csv')
test = pd.read_csv(PATH + 'test_clean.csv')

train.head()

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
def add_category(train, test):
    train["category"] = 0
    test["category"] = 0
    
    # train segments with more then 9 open channels classes
    train.loc[2_000_000:2_500_000-1, 'category'] = 1
    train.loc[4_500_000:5_000_000-1, 'category'] = 1
    
    # test segments with more then 9 open channels classes (potentially)
    test.loc[500_000:600_000-1, "category"] = 1
    test.loc[700_000:800_000-1, "category"] = 1
    
    return train, test

#train, test = add_category(train, test)

In [ ]:
%%time

# create batches of GROUP_BATCH_SIZE observations
def batching(df, batch_size, gr_name='group'):
    df[gr_name] = df.groupby(df.index//batch_size, sort=False)['signal'].agg(['ngroup']).values
    df[gr_name] = df[gr_name].astype(np.uint16)
    return df

# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    train_input_mean = train.signal.values.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test

def run_feat_engineering(df, batch_size, gr_name='group'):
    df = batching(df, batch_size = batch_size, gr_name=gr_name)
    df['signal_2'] = df['signal'] ** 2
    df['signal_2-7500-mean'] = df['signal_2'] - df['signal_2'].rolling(window=7500).mean()    
    #df['signal_2-mean'] = df['signal_2'] - df['signal_2'].mean()
    return df

#train, test = normalize(train, test)
train = run_feat_engineering(train, batch_size = GROUP_BATCH_SIZE1, gr_name='group1')
train = batching(train, batch_size = GROUP_BATCH_SIZE2, gr_name='group2')

test = run_feat_engineering(test, batch_size = GROUP_BATCH_SIZE1, gr_name='group1')
test = batching(test, batch_size = GROUP_BATCH_SIZE2, gr_name='group2')

In [ ]:
%%time

## add some noise

STD = 0.01

old_data = train['signal']
new_data = old_data + np.random.normal(0,STD,size=len(train)) 
train['signal'] = new_data

old_data = test['signal']
new_data = old_data + np.random.normal(0,STD,size=len(test)) 
test['signal'] = new_data

del old_data, new_data

In [ ]:
%%time

def gen_roll_features(full, win_sizes = WINDOW_SIZES):
    for window in tqdm(win_sizes):
        full["rolling_mean_" + str(window)] = full['signal'].rolling(window=window).mean()
        full["rolling_std_" + str(window)] = full['signal'].rolling(window=window).std()
        full["rolling_var_" + str(window)] = full['signal'].rolling(window=window).var()
        full["rolling_min_" + str(window)] = full['signal'].rolling(window=window).min()
        full["rolling_max_" + str(window)] = full['signal'].rolling(window=window).max()

        a = (full['signal'] - full['rolling_min_' + str(window)]) / (full['rolling_max_' + str(window)] - full['rolling_min_' + str(window)])
        full["norm_" + str(window)] = a * (np.floor(full['rolling_max_' + str(window)]) - np.ceil(full['rolling_min_' + str(window)]))
    return full

train = gen_roll_features(train)
test = gen_roll_features(test)

In [ ]:
%%time

def gen_sig_features(df):
    df = df.sort_values(by=['time']).reset_index(drop=True)
    df.index = ((df.time * 10_000) - 1).values
    df['batch'] = df.index // 25_000
    df['batch_index'] = df.index  - (df.batch * 25_000)
    df['batch_slices'] = df['batch_index']  // 2500
    df['batch_slices2'] = df.apply(lambda r: '_'.join([str(r['batch']).zfill(3), str(r['batch_slices']).zfill(3)]), axis=1)
    
    for c in tqdm(['batch','batch_slices2']):
        d = {}
        d['mean'+c] = df.groupby([c])['signal'].mean()
        d['median'+c] = df.groupby([c])['signal'].median()
        d['max'+c] = df.groupby([c])['signal'].max()
        d['min'+c] = df.groupby([c])['signal'].min()
        d['std'+c] = df.groupby([c])['signal'].std()
        d['mean_abs_chg'+c] = df.groupby([c])['signal'].apply(lambda x: np.mean(np.abs(np.diff(x))))
        d['abs_max'+c] = df.groupby([c])['signal'].apply(lambda x: np.max(np.abs(x)))
        d['abs_min'+c] = df.groupby([c])['signal'].apply(lambda x: np.min(np.abs(x)))
        d['range'+c] = d['max'+c] - d['min'+c]
        d['maxtomin'+c] = d['max'+c] / d['min'+c]
        d['abs_avg'+c] = (d['abs_min'+c] + d['abs_max'+c]) / 2
        for v in d:
            df[v] = df[c].map(d[v].to_dict())
    df = reduce_mem_usage(df)
    gc.collect()
    return df

train = gen_sig_features(train)
test = gen_sig_features(test)

In [ ]:
%%time

def gen_shift_features(df):
    # add shifts
    df['signal_shift_+1'] = [0,] + list(df['signal'].values[:-1])
    df['signal_shift_-1'] = list(df['signal'].values[1:]) + [0]
    for i in df[df['batch_index']==0].index:
        df['signal_shift_+1'][i] = np.nan
    for i in df[df['batch_index']==49999].index:
        df['signal_shift_-1'][i] = np.nan
    
    df['signal_shift_+2'] = [0,] + [1,] + list(df['signal'].values[:-2])
    df['signal_shift_-2'] = list(df['signal'].values[2:]) + [0] + [1]
    for i in df[df['batch_index']==0].index:
        df['signal_shift_+2'][i] = np.nan
    for i in df[df['batch_index']==1].index:
        df['signal_shift_+2'][i] = np.nan
    for i in df[df['batch_index']==49999].index:
        df['signal_shift_-2'][i] = np.nan
    for i in df[df['batch_index']==49998].index:
        df['signal_shift_-2'][i] = np.nan
    
    df.drop(columns=['batch', 'batch_index', 'batch_slices', 'batch_slices2'], inplace=True)
    gc.collect()

    for c in [c1 for c1 in df.columns if c1 not in ['time', 'signal', 'open_channels', 'group1', 'group2', 'category', 'index']]:
        df[c+'_msignal'] = df[c] - df['signal']
        
    df = df.replace([np.inf, -np.inf], np.nan)    
    df.fillna(0, inplace=True)
    df = reduce_mem_usage(df)
    gc.collect()
    return df

train = gen_shift_features(train)
test = gen_shift_features(test)

In [ ]:
BEST_FEATURES = [
    'signal',
    'signal_2', 
    'signal_2-7500-mean',
    'minbatch_slices2_msignal',
    'rolling_max_1000_msignal',
    'signal_shift_-2_msignal',
    'rangebatch_msignal',
    'medianbatch_slices2_msignal',
    'signal_shift_-1_msignal',
    'mean_abs_chgbatch_slices2',
    'abs_avgbatch_msignal',
    'signal_shift_+1_msignal',
    'rolling_min_10_msignal',
    'abs_maxbatch_msignal',
    'minbatch_msignal',
    'mean_abs_chgbatch',
    'abs_minbatch',
    'signal_2_msignal',
    'maxbatch_slices2_msignal',
    'rolling_var_1000_msignal',
    'rolling_min_1000',
    'rolling_max_500_msignal',
    'norm_1000_msignal',
    'rolling_mean_10_msignal',
    'abs_minbatch_slices2_msignal',
    'rolling_min_50_msignal',
    'rolling_min_500',
    'rangebatch_slices2',
    'abs_minbatch',
    'meanbatch_slices2',
    'abs_minbatch_msignal',
    'minbatch_slices2_msignal',
    'signal_shift_+2_msignal',
    'signal_shift_-2_msignal',
    'mean_abs_chgbatch_msignal',
    'rolling_var_3_msignal',
    'signal_shift_-2',
    'signal_shift_+1',
    'rolling_max_5000',
    'rolling_var_5000',
    'rolling_max_100',
]    

print(len(BEST_FEATURES))
FEATURES = [c for c in train.columns if c not in ['index, time', 'open_channels', 'group1', 'group2', 'index_msignal' ]]

In [ ]:
%%time
#from sklearn.decomposition import PCA

#all_other_cols = [c for c in FEATURES if c not in BEST_FEATURES]

#pca = PCA(n_components=1)
#pca_train = pca.fit_transform(train[all_other_cols])
#pca_test = pca.transform(test[all_other_cols])

#train['pca_1'] = pca_train[:,0]
#test['pca_1'] = pca_test[:,0]
#BEST_FEATURES.append('pca_1')

#del pca_train, pca_test, pca

#y_train = train['open_channels']
#train.drop(all_other_cols, inplace=True, axis=1)
#test.drop(all_other_cols, inplace=True, axis=1)
#gc.collect()

## LGB Model

In [ ]:
## reduce amount of data to speed things up

#SAMPLE_RATE = 2

y_train = train['open_channels']
X = train[BEST_FEATURES]#[::SAMPLE_RATE]
y = y_train#[::SAMPLE_RATE]

groups1 = train['group1']#[::SAMPLE_RATE]
groups2 = train['group2']#[::SAMPLE_RATE]

X_test = test[BEST_FEATURES]
gc.collect()

print(f'Original sizes: train: {train.shape}, y_train: {y_train.shape}' )
print(f'Original sizes: test: {test.shape}' )
print(f'Reduced train sizes: X: {X.shape}, y: {y.shape},  X_test: {X_test.shape}' )
print(f'Reduced test sizes: X_test: {X_test.shape}' )


In [ ]:
%%time

#scaler = RobustScaler(copy=False)

#scaler.fit(X[BEST_FEATURES].values)

#X[BEST_FEATURES] = scaler.transform(X[BEST_FEATURES].to_numpy()) 
#X_test[BEST_FEATURES] = scaler.transform(X_test[BEST_FEATURES].to_numpy())

In [ ]:
%%time

NUM_BOOST_ROUND = 2000 
EARLY_STOPPING_ROUNDS = 100
VERBOSE_EVAL = 100
LEARNING_RATE = 0.05
MAX_DEPTH = -1
NUM_LEAVES = 250

TOTAL_SPLITS = 6

lgb_params2 = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 11,
    'metric': 'multi_logloss',
    'learning_rate': 0.01987173774816051,
    'lambda_l1': 0.00031963798315506463,
    'lambda_l2': 0.18977456778807847,
    'num_leaves': 171, 
    'feature_fraction': 0.58733782457345, 
    'bagging_fraction': 0.7057826081907392, 
    'bagging_freq': 4,
    'seed': RANDOM_SEED,
    #'max_depth': 73,
    #'colsample_bytree': 0.6867118652742716,
}

def run_for_group(groups):    
    feat_imp_ = pd.DataFrame(np.zeros((len(BEST_FEATURES), 2)), columns=['vals','names'])
    feat_imp_.names = BEST_FEATURES
    oof_ = pd.DataFrame(np.zeros((len(X), 1))) 
    preds_ = pd.DataFrame(np.zeros((len(X_test), 1)))
    kf = GroupKFold(n_splits=TOTAL_SPLITS)

    fold = 1
    for tr_idx, val_idx in kf.split(X, y, groups=groups):
        print(f'====== Fold {fold:0.0f} of {TOTAL_SPLITS} ======')
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model = lgb.train(
            lgb_params2, valid_names=["train", "valid"], 
            train_set=lgb.Dataset(X_tr, y_tr ), # 
            num_boost_round = NUM_BOOST_ROUND,
            valid_sets = [lgb.Dataset(X_val, y_val)], #  
            verbose_eval = VERBOSE_EVAL,
            early_stopping_rounds = EARLY_STOPPING_ROUNDS,
            feval = multiclass_F1_Metric) ##MacroF1Metric

        feat_imp_.vals += model.feature_importance() 

        oof = model.predict(X_val, num_iteration=model.best_iteration)
        oof = np.argmax(oof, axis=1).reshape(-1)
        test_preds = model.predict(X_test, num_iteration=model.best_iteration)
        test_preds = np.argmax(test_preds, axis=1).reshape(-1)

        oof_.iloc[oof_.index.isin(val_idx)] = oof.reshape(-1,1)
        preds_ += test_preds.reshape(-1,1)

        print( f'OOF F1 score: {f1_score(y_val, oof, average = "macro")}') 
        print( f'OOF RMSE: {np.sqrt(mean_squared_error(y_val, oof))}') 
        fold = fold+1

    return preds_ / TOTAL_SPLITS, oof_, feat_imp_

yhat_group1, oof1, fi1 = run_for_group(groups1)
yhat_group2, oof2, fi2 = run_for_group(groups2)

====== Fold 1 of 5 ======
Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.635213	train's MacroF1Metric: 0.936415
[200]	train's multi_logloss: 0.286776	train's MacroF1Metric: 0.936632
[300]	train's multi_logloss: 0.163175	train's MacroF1Metric: 0.936808
[400]	train's multi_logloss: 0.11635	train's MacroF1Metric: 0.936916
Early stopping, best iteration is:
[327]	train's multi_logloss: 0.145912	train's MacroF1Metric: 0.936955

In [ ]:
fi1.sort_values(by='vals', inplace=True, ascending=False)
fi1.head(33)

In [ ]:
fi2.sort_values(by='vals', inplace=True, ascending=False)
fi2.head(33)

In [ ]:
gc.collect()

## Evaluation

In [ ]:
f1_lgb1 = f1_score(np.round(np.clip(oof1, 0, 10)).astype(int), y, average='macro')
print(f'OOF FULL F1MACRO: {f1_lgb1}')
f1_lgb2 = f1_score(np.round(np.clip(oof2, 0, 10)).astype(int), y, average='macro')
print(f'OOF FULL F1MACRO: {f1_lgb2}')

y_pred = np.round(np.clip((oof1+oof2)/2, 0, 10)).astype(int).values
optR = OptimizedRounder()
optR.fit(y_pred.reshape(-1,), y)

opt_pred = optimize_predictions(y_pred, optR.coefficients()).astype(int)
opt_f1 = f1_score(opt_pred, y, average='macro')
print(f'OPTIMIZED LGB F1MACRO: {opt_f1}')

In [ ]:
plot_cm(y, opt_pred, 'LGB Confusion Matrix')

## Submission

In [ ]:
y_pred = np.round(np.clip((yhat_group1+yhat_group2)/2, 0, 10)).astype(int).values
opt_pred = optimize_predictions(y_pred, optR.coefficients()).astype(int)

sub = pd.read_csv('../input/liverpool-ion-switching/sample_submission.csv')
sub['open_channels'] = np.round(np.clip(opt_pred, 0, 10)).astype(int)
sub.to_csv('submission.csv', index=False, float_format='%.4f')

sub.head(20)